In [2]:
# 系数特征编码
# 针对用户侧和新闻侧的稀疏特征进行编码，并将训练样本join上两侧的特征

# 数据和测试数据
data, user_data, doc_data = get_all_data()

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`
feature_max_idx = {}
feature_encoder = {}

user_sparse_features = ["user_id", "device", "os", "province", "city", "age", "gender"]
for feature in user_sparse_features:
    lbe = LabelEncoder()
    user_data[feature] = lbe.fit_transform(user_data[feature]) + 1
    feature_max_idx[feature] = user_data[feature].max() + 1
    feature_encoder[feature] = lbe


doc_sparse_features = ["article_id", "cate", "sub_cate"]
doc_dense_features = ["title_len", "img_num"]

for feature in doc_sparse_features:
    lbe = LabelEncoder()
    if feature in ["cate","sub_cate"]:
        # 这里面会出现一些float的数据，导致无法编码
        doc_data[feature] = lbe.fit_transform(doc_data[feature].astype(str)) + 1
    else:
        doc_data[feature] = lbe.fit_transform(doc_data[feature]) + 1
    feature_max_idx[feature] = doc_data[feature].max() + 1
    feature_encoder[feature] = lbe

data["article_id"] = feature_encoder["article_id"].transform(data["article_id"].tolist())
data["user_id"] = feature_encoder["user_id"].transform(data["user_id"].tolist())


# join 用户侧和新闻侧的特征
data = data.join(user_data.set_index("user_id"), on="user_id", how="inner")
data = data.join(doc_data.set_index("article_id"), on="article_id", how="inner")

sparse_features = user_sparse_features + doc_sparse_features
dense_features = doc_dense_features

features = sparse_features + dense_features

mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])


NameError: name 'get_all_data' is not defined

In [ ]:
# 配置特征以及模型训练
# 构建模型所需要的输入特征，同时构建DSSM模型并且训练
from model import DSSM
embedding_dim = 8
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
                        SparseFeat("gender", feature_max_idx['gender'], embedding_dim),
                        SparseFeat("age", feature_max_idx['age'], embedding_dim),
                        SparseFeat("device", feature_max_idx['device'], embedding_dim),
                        SparseFeat("os", feature_max_idx['os'], embedding_dim),
                        SparseFeat("province", feature_max_idx['province'], embedding_dim),
                        SparseFeat("city", feature_max_idx['city'], embedding_dim),  ]

item_feature_columns = [SparseFeat('article_id', feature_max_idx['article_id'], embedding_dim),
                        DenseFeat('img_num', 1),
                        DenseFeat('title_len', 1),
                        SparseFeat('cate', feature_max_idx['cate'], embedding_dim),
                        SparseFeat('sub_cate', feature_max_idx['sub_cate'], embedding_dim)]

model = DSSM(user_feature_columns, item_feature_columns,
          user_dnn_hidden_units=(32, 16, embedding_dim), item_dnn_hidden_units=(32, 16, embedding_dim))  # FM(user_feature_columns,item_feature_columns)

model.compile(optimizer="adagrad", loss = "binary_crossentropy", metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()] ) #

history = model.fit(train_model_input, train_label, batch_size=256, epochs=4, verbose=1, validation_split=0.2, )

In [ ]:
# 生成embedding用于召回
# 利用巡礼过的模型获取所有item的embeddings，同时获取所有测试集的user embedding
all_item_model_input = {"article_id": item_profile['article_id'].values,
                        "img_num": item_profile['img_num'].values,
                        "title_len": item_profile['title_len'].values,
                        "cate": item_profile['cate'].values,
                        "sub_cate": item_profile['sub_cate'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

user_idx_2_rawid, doc_idx_2_rawid = {}, {}

for i in range(len(user_embs)):
    user_idx_2_rawid[i] = test_user_model_input["user_id"][i]

    for i in range(len(item_embs)):
        doc_idx_2_rawid[i] = all_item_model_input["article_id"][i]

        # 保存一份
        pickle.dump((user_embs, user_idx_2_rawid, feature_encoder["user_id"]), open(file_path + 'user_embs.pkl', 'wb'))
        pickle.dump((item_embs, doc_idx_2_rawid, feature_encoder["article_id"]), open(file_path + 'item_embs.pkl', 'wb'))
